In [125]:
import tensorflow as tf
import numpy as np
import os
import re
import io
import librosa
import pickle
import random
import matplotlib.pyplot as plt
import IPython.display as ipd
from os.path import isdir, join
%matplotlib inline
import time

In [2]:
save_dir = "/home/maikfangogoair/tmp/save/"
pkl_file = "spect_v25.pkl"
with open (save_dir + pkl_file, 'rb') as fp:
    all_data = pickle.load(fp)
print("There are total %d data to be trained." % len(all_data))

There are total 97043 data to be trained.


In [126]:
#https://github.com/philipperemy/tensorflow-ctc-speech-recognition/blob/master/ctc_tensorflow_example.py

In [135]:

# Accounting the 0th index + blank label = 27 characters
num_classes = ord('z') - ord('a') + 1 + 1
FIRST_INDEX = ord('a') - 1  # 0 is reserved to space

# Hyper-parameters
num_layers = 1
num_features = 161
num_hidden = 400


In [136]:
graph = tf.Graph()
with graph.as_default():
    # [batch_size, max_step_size, num_features]
    inputs = tf.placeholder(tf.float32, [None, None, num_features])
    seq_len = tf.placeholder(tf.int32, [None])
    cell = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)
    stack = tf.contrib.rnn.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
    outputs, _ = tf.nn.dynamic_rnn(stack, inputs, seq_len, dtype=tf.float32)
    shape = tf.shape(inputs)
    batch_s, max_time_steps = shape[0], shape[1]
    outputs = tf.reshape(outputs, [-1, num_hidden])
    logits = tf.layers.dense(outputs, num_classes)
    #batch_size, time, num_classes
    logits = tf.reshape(logits, [batch_s, -1, num_classes])
    #time, batch_size, num_classes
    logits = tf.transpose(logits, (1, 0, 2))
    targets = tf.sparse_placeholder(tf.int32)
    loss = tf.nn.ctc_loss(targets, logits, seq_len)
    cost = tf.reduce_mean(loss)
    learning_rate_input = tf.placeholder(tf.float32, [], name='learning_rate_input')
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate_input, momentum=0.9).minimize(cost)
    decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, seq_len)
    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),targets))

In [137]:
def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n] * len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int64)

    return indices, values, shape

In [138]:

def pad_sequences(sequences, maxlen=None, dtype=np.float32,
                  padding='post', truncating='post', value=0.):
    '''Pads each sequence to the same length: the length of the longest
    sequence.
        If maxlen is provided, any sequence longer than maxlen is truncated to
        maxlen. Truncation happens off either the beginning or the end
        (default) of the sequence. Supports post-padding (default) and
        pre-padding.
        Args:
            sequences: list of lists where each element is a sequence
            maxlen: int, maximum length
            dtype: type to cast the resulting sequence.
            padding: 'pre' or 'post', pad either before or after each sequence.
            truncating: 'pre' or 'post', remove values from sequences larger
            than maxlen either in the beginning or in the end of the sequence
            value: float, value to pad the sequences to the desired value.
        Returns
            x: numpy array with dimensions (number_of_sequences, maxlen)
            lengths: numpy array with the original sequence lengths
    '''
    lengths = np.asarray([len(s) for s in sequences], dtype=np.int64)

    nb_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((nb_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if len(s) == 0:
            continue  # empty list was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x, lengths

In [ ]:
val_set = [x for x in all_data if x["group"] == "validation"]
train_set = [x for x in all_data if x["group"] == "training"]
print("val_set size: %d, train_set size: %d" % (len(val_set),len(train_set)))

display_interval = 20
val_interval = 100
save_interval = 200
learning_rate = 1e-4
batch_size = 100
save_dir = "/home/maikfangogoair/tmp/save/"
steps=20000
train_size = len(train_set)
val_size = len(val_set)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    saver = tf.train.Saver(tf.global_variables())
    saver.restore(session, save_dir + 'RNN_CTC.ckpt-3200')
    val_pos = 0
    for batch in range(steps):
        pos = batch * batch_size % train_size
        train_inputs = np.squeeze(np.asarray([x["feature"] for x in train_set[pos:min(pos+batch_size, train_size)]]), axis=1).transpose([0,2,1])
        batch_train_inputs = train_inputs
        # Padding input to max_time_step of this batch
        batch_train_inputs, batch_train_seq_len = pad_sequences(batch_train_inputs)

        tmp1 = []
        for x in train_set[pos:pos+batch_size]:
            tmp1.append(np.asarray([ord(y) - FIRST_INDEX for y in x["label"]]))
        batch_train_targets=np.asarray(tmp1)
        batch_train_targets = sparse_tuple_from(batch_train_targets)
        feed = {inputs: batch_train_inputs,
                targets: batch_train_targets, \
                seq_len: batch_train_seq_len, \
                learning_rate_input: learning_rate }

        train_cost, _ = session.run([cost, optimizer], feed)
        train_ler = session.run(ler, feed_dict=feed)

        if batch % display_interval == 0 :
            print("step %d, train_cost %f, train_ler: %f" % (batch, train_cost, train_ler))



        if batch % val_interval == 0 :
            val_pos = batch // val_interval * batch_size % val_size
            
            val_pos = val_pos % len(val_set)
            batch_val_inputs = np.squeeze(np.asarray([x["feature"] for x in val_set[val_pos:min(val_pos+batch_size, val_size)]]), axis=1).transpose([0,2,1])
            # Padding input to max_time_step of this batch
            batch_val_inputs, batch_val_seq_len = pad_sequences(batch_val_inputs)
            tmp2 = []
            for x in val_set[val_pos:val_pos+batch_size]:
                tmp2.append(np.asarray([ord(y) - FIRST_INDEX for y in x["label"]]))
            batch_val_targets=np.asarray(tmp2)
            batch_val_targets = sparse_tuple_from(batch_val_targets)
            val_feed = {inputs: batch_val_inputs,
                        targets: batch_val_targets,
                        seq_len: batch_val_seq_len}

            val_cost, val_ler = session.run([cost, ler], feed_dict=val_feed)
            print("val step %d, val_cost %f, val_ler: %f" % (batch // val_interval, val_cost, val_ler))

        if  batch % save_interval == 0:
            saver.save(session, save_dir + "RNN_CTC.ckpt", global_step=batch)


val_set size: 10132, train_set size: 80756
INFO:tensorflow:Restoring parameters from /home/maikfangogoair/tmp/save/RNN_CTC.ckpt-3200


INFO:tensorflow:Restoring parameters from /home/maikfangogoair/tmp/save/RNN_CTC.ckpt-3200


step 0, train_cost 5.158530, train_ler: 0.498333
val step 0, val_cost 4.726611, val_ler: 0.443810
step 20, train_cost 5.650603, train_ler: 0.533571
step 40, train_cost 4.799384, train_ler: 0.493190
step 60, train_cost 4.838092, train_ler: 0.447857
step 80, train_cost 5.716147, train_ler: 0.568929
step 100, train_cost 5.479010, train_ler: 0.493667
val step 1, val_cost 5.896012, val_ler: 0.481905
step 120, train_cost 4.569933, train_ler: 0.467024
step 140, train_cost 5.715308, train_ler: 0.470024
step 160, train_cost 5.207690, train_ler: 0.468857
step 180, train_cost 5.385999, train_ler: 0.514143
step 200, train_cost 5.368464, train_ler: 0.556548
val step 2, val_cost 5.222622, val_ler: 0.458548
step 220, train_cost 5.757406, train_ler: 0.528691
step 240, train_cost 5.052781, train_ler: 0.453310
step 260, train_cost 4.975933, train_ler: 0.512500
step 280, train_cost 4.828191, train_ler: 0.484524
step 300, train_cost 4.688286, train_ler: 0.457143
val step 3, val_cost 4.784322, val_ler: 0.4